In [1]:
import json
import pandas as pd

with open("etiketli_yorumlar_tinyllama.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data["reviews"])
print(df.head())


                                             comment  label
0  Harika tek kelimeyle çok iyi bir bisiklet<br>F...      0
1     Ürün çok güzel sadece koltuk biraz fazla sert.      1
2  Önceden mtb kullandığımdan frenler yetersiz hi...      0
3  2 yıldır kullanıyorum hiçbir sorun vermedi, şe...      0
4  bisikleti ilk aldığımda frenlere pek alışamamı...      1


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Özellik ve hedef
X_text = df["comment"]
y = df["label"]

# TF-IDF vektörleştirici
vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1, 2))
X = vectorizer.fit_transform(X_text)

# Eğitim/test bölünmesi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model eğitimi
model = LogisticRegression(max_iter=1000, class_weight="balanced")
model.fit(X_train, y_train)

# Değerlendirme
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, digits=3))


              precision    recall  f1-score   support

           0      0.750     0.462     0.571        13
           1      0.462     0.750     0.571         8

    accuracy                          0.571        21
   macro avg      0.606     0.606     0.571        21
weighted avg      0.640     0.571     0.571        21



In [6]:
def yorum_tahmin_et(model, vectorizer, yorumlar):
    X_yeni = vectorizer.transform(yorumlar)
    tahminler = model.predict(X_yeni)
    etiket_map = {-1: "Negatif", 0: "Nötr", 1: "Pozitif"}
    return [(yorum, etiket_map[t]) for yorum, t in zip(yorumlar, tahminler)]

# Örnek yeni yorumlar
yeni_yorumlar = [
    "Ürün kötü çıktı, iade ettim.",
    "Beklediğimden iyi, tavsiye ederim",
    "Orta kalite, iş görür",
    "Ürün çok iyi",
    "Ürün çok kötü"
]

print(yorum_tahmin_et(model, vectorizer, yeni_yorumlar))


[('Ürün kötü çıktı, iade ettim.', 'Nötr'), ('Beklediğimden iyi, tavsiye ederim', 'Pozitif'), ('Orta kalite, iş görür', 'Pozitif'), ('Ürün çok iyi', 'Nötr'), ('Ürün çok kötü', 'Nötr')]
